# What drives the price of a car?

In [360]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.pipeline import Pipeline

## Business Understanding


So, the goal of this project is to find a number of metrics that drive customers to purchase cars. Our client is going to be a used car dealership, so we want to generate a number of factors that are important to the price of used cars. The data is provided so there is no need to acquire data. This project will likely involve cleaning and preparing the dataset, then proceeding to create regression models to find the key drivers of used car prices.

## Data Understanding

To start, I need to get a sense of the dataset, including whether the data is numeric or not. I need to analyse the dataset and perform any data cleaning to ensure that the data we have is useable in our future models. This begins with an understanding of the dataset. I'll start by loading the data and looking at a header to get a sense of what I'm dealing with.

In [257]:
data = pd.read_csv('data/vehicles.csv')

In [258]:
data.dropna().head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,7305672709,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,7305672266,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,7305672252,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,7316482063,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,7316429417,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


I can already see some redundant data and some irrelevant data. I dropped all null values in the header so that I could see what types of data I had, but since the index does not start at 1 after the drop, there is an issue of null/blank values. 

Some of this data is also not relevant, like the VIN numbers. The VIN numbers do contain some info in them, but they are unique to each car and too heavily formatted to be easily parsed for my model. This means that I am going to need to drop some columns and some rows to get a clean dataset.

I want to know how many null values I have in this dataset, so I will check the non-null values:

In [259]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

There is a significant amount of empty cells, and this might compromise my dataset. I am going to do some testing to see what I can drop to get a cleaner dataset. My goal is to keep around 50% of the rows from my cleaning to ensure my analysis is robust.

In [260]:
# Testing a naive approach and dropping all null values.
drop_rows_data = data.dropna()
drop_percent = round(drop_rows_data.shape[0]/data.shape[0]*100, 2)
print(f"\nRemaining rows after drop {drop_percent}%")


Remaining rows after drop 8.17%


Dropping the rows with null values leaves only 8% of the data remaining. This is far too significant of a loss of data, so I'll try and drop columns before we drop the rows. I will drop the columns that I think are unrelated or redundant first. After that, I will remove columns by ordering them by how much of their data is null or NaN and rmeoving them in order. I will stop when the data is near or above 50%.

In [261]:
# Testing adding columns by how much data they are missing.
drop_columns = data.drop(['size', 'cylinders', 'VIN', 'state', 'region', 'model', 'condition', 'id'], axis = 1).dropna()
drop_percent2 = round(drop_columns.shape[0]/data.shape[0]*100, 2)
print(f"\nRemaining rows after drop {drop_percent2}%")


Remaining rows after drop 48.59%


Finally I have a dataset that is large enough to perform robust analysis. At this point, I am ready to prepare the data for preparation.

## Data Preparation

I will use the "drop columns" dataset that I saved from the previous step as my basis for my data preparation.

In [263]:
# Resetting index and displaying a header to analyze next steps
drop_columns = drop_columns.reset_index(drop = True)
drop_columns.head()

,price,year,manufacturer,fuel,odometer,title_status,transmission,drive,type,paint_color
0,15000,2013.0,ford,gas,128000.0,clean,automatic,rwd,truck,black
1,27990,2012.0,gmc,gas,68696.0,clean,other,4wd,pickup,black
2,34590,2016.0,chevrolet,gas,29499.0,clean,other,4wd,pickup,silver
3,35000,2019.0,toyota,gas,43000.0,clean,automatic,4wd,truck,grey
4,29990,2016.0,chevrolet,gas,17302.0,clean,other,4wd,pickup,red


Now I need to perform any steps to get the data into a form that is ready for modeling. The most important of which is to ensure that my data is readable by my models. Looking at the data, I have two problems that need to be solved: the nominal data needs to be numeric, and the numeric data is not standardized. I will use a one hot encoding strategy on the non-numeric columns to prepare them for modeling. For the remaining numeric data, I will scale the data.

In [354]:
# First I need to separate the target feature and the remaining features into new temporary dataframes.
temp_X = drop_columns.drop(['price'], axis = 1)
temp_y = drop_columns['price']

# First I split the data that is being scaled and the data that is being one hot encoded.
numeric_X = temp_X[['odometer', 'year']]                       # This data needs will be scaled
nominal_X = temp_X.drop(['odometer', 'year'], axis = 1)        # This data will be encoded

# Now I scale the numeric data
scaled_X = (numeric_X - numeric_X.mean())/(numeric_X.std())
y = (y - y.mean())/(y.std())

# Next I need to use a one hot encoding strategy on the nominal data
encoded_X = pd.get_dummies(nominal_X)

# Now, I merge the encoded and scaled data back together to form one dataframe
X = pd.concat([scaled_X, encoded_X], axis = 1)

# Finally I'm going to split the data into train and test sets, in anticipation of modeling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

Now I have my data prepared for modeling. The dataset is numeric, and the continuous features are scaled. Note that the data is largely binary, with high dimensionality. This means that a feature reduction or linear combination of features would not be advisable. I can still perform reductions in the models, but this may cause more features to be dropped from analysis.

## Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Simple Linear Regression

To start, I run a simple linear regression on the data to get a baseline for future models and to get a better sense of the data.

In [356]:
model = LinearRegression().fit(X_train, y_train)

train_mse = mean_squared_error(model.predict(X_train), y_train)
test_mse = mean_squared_error(model.predict(X_test), y_test)

print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

Train MSE: 0.12013834308653984
Test MSE: 3.052925210219233


The simple linear regression model performed much better on the test mse and gives us a baseline of what is happening with the models. The data has a large number of features, so this would be a good time to do a Sequential Feature Selection to reduce the dimensionality and pull out a small number of features that are most improtant to the model.

### Sequential Feature Selection

In [357]:
selector = SequentialFeatureSelector(estimator = LinearRegression(), n_features_to_select=6, cv = 5)
best_features = selector.fit_transform(X, y)
best_features_df = pd.DataFrame(best_features, columns = selector.get_feature_names_out())

In [358]:
best_features_df.head()

,manufacturer_aston-martin,manufacturer_datsun,manufacturer_ferrari,manufacturer_harley-davidson,manufacturer_land rover,manufacturer_morgan
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


Next I will apply the train and test data to the model and see how it performs in a linear regression.

In [359]:
# Here is a simple linear regression on the selected features
linreg = LinearRegression().fit(X_train, y_train)
train_mse = mean_squared_error(linreg.predict(X_train), y_train)
test_mse = mean_squared_error(linreg.predict(X_test), y_test)
print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

Train MSE: 0.12013834308653984
Test MSE: 3.052925210219233


In the next model, I will use GridSearchCV to optimize hyperparameters.

### GridSearchCV with Ridge Regression

For this model, I want to use a GridSearchCV to optimize hyperparameters. In this case, I want to find the best alpha value for my ridge regression. I am using the default 5-fold cross validation in this case.

In [347]:
# I will test three alpha values for this model.
ridge_param_dict = {'ridge__alpha': [1, 0.1, 10]}
ridge_pipe = Pipeline([('ridge', Ridge())])
ridge_grid = GridSearchCV(ridge_pipe, param_grid= ridge_param_dict).fit(X_train, y_train)

# Now that I have a fit pipeline, I will extract the best model from the pipeline and calculate some descriptive charecteristics
best_estimator = ridge_grid.best_estimator_
best_selector = best_estimator.named_steps['ridge']
best_model = best_estimator.named_steps['ridge']
best_params = ridge_grid.best_params_

GS_train_mse = mean_squared_error(best_model.predict(X_train), y_train)
GS_test_mse = mean_squared_error(best_model.predict(X_test), y_test)

# Here I will print out some data so that I can analyze the results.
print(best_params)
#print(best_estimator[:-1].get_feature_names_out(X_train))
print(f"Train MSE: {GS_train_mse}")
print(f"Test MSE: {GS_test_mse}")

{'ridge__alpha': 1}
Train MSE: 1.3685110667947205
Test MSE: 0.25180959459527064


With this, I now know the optimal alpha is 1, which is the default for GridSearchCV. Now I will build a model to find the optimal number of features with the best alpha.

### GridSearchCV for SFS hyperparameters

In [346]:
#I will test for the optimal number of features selected
selector_pipe = Pipeline([('selector', SequentialFeatureSelector(LinearRegression())),
                         ('ridge', Ridge())])
param_dict = {'selector__n_features_to_select': [2, 3, 4]}
selector_grid = GridSearchCV(selector_pipe, param_grid = param_dict).fit(X_train, y_train)

#Now i will calculate some descriptive variables to better understand the results.
best_estimator = selector_grid.best_estimator_
best_selector = best_estimator.named_steps['selector']
best_model = best_estimator.named_steps['ridge']
best_params = selector_grid.best_params_
feature_names = X_train.columns[best_selector.get_support()]

selector_train_mse = mean_squared_error(selector_grid.predict(X_train), y_train)
selector_test_mse = mean_squared_error(selector_grid.predict(X_test), y_test)

print(best_params)
print(feature_names)
print(f'Train MSE: {selector_train_mse}')
print(f'Test MSE: {selector_test_mse}')

{'selector__n_features_to_select': 3}
Index(['manufacturer_aston-martin', 'manufacturer_ferrari',
       'manufacturer_morgan'],
      dtype='object')
Train MSE: 1.3685110717611002
Test MSE: 0.25180959861813135


After this model, we have found the optimal hyperparameters and the optimal columns for the regression.

## Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

Looking at the results of these models, there is a clear pattern of overfitting. The train MSE is consistently much higher than the test MSE. Having a low test MSE is good, but the difference is massive. There is also the issue that the MSE for each model is too similar. I think this is due to the same train/test split being used, but with further testing I've found that there is alot of variance depending on the split of the data. This high variance reinforces my idea that overfitting is biasing the results.

This means that further testing could be applied to potentially further reduce the overfitting and produce a more accurate model. In this case, we can still perform analysis on the moels and see what can be learned. The models were able to achieve a low MSE which means they were able to predict the price fairly well form the features. Since the data is normalized and it has high dimensionality, it would be difficult to look at a table of coefficients and find which factors are the most important. Instead, I noticed that every feature selection program found that the best predictors of price were all encodings of the "manufacturer" column. This follows common sense, as it is commonly known that manufacturers like Aston-Martin and Ferrari produce luxury cars that cost orders of magnitude more than brands like Fords.

Combining the overfitting of the model, and the consistent selection of "manufacturer" features, we can conclude that this model does not bring a lot of valuable information to the client. We can safely say that the manufacturer is vital for determining price, but this model is not going to bring any deeper insights beyond that. Potentially revisiting the data nds olving the overfitting problem could bring more useful insights. Removing the manufacturer column and adding back some other columns may provide a more unique insight tha tmay be less apparent.

## Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

Overall, I would say that the most important factor in my car pricing model is the Manufacturer. Other details like age and mileage don't compare in importance to the manufacturer of the car. This model has shown that the dealer would do well to rely on brand recognition, and to price their cars according to the manufacturer. Also note that this model is not infallible, and even though the model has shown the value of the manufacturer in determining price, there is a limit to what a computer model can predict, and a dealer should use their best judgement on pricing. This information is limited by the dataset, which may not encompass the reality of cars sold on a lot.